<a href="https://colab.research.google.com/github/james-monahan/Code-school-notebooks/blob/main/Week-14-nlp-regex/Natural_Language_Processing_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyforest

In [2]:
from pyforest import *

In [3]:
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [ ]:
import nltk
nltk.download('popular')

In [5]:
link = "https://github.com/murpi/wilddata/raw/master/quests/tweets.zip"
df = pd.read_csv(link, compression='zip') 

<IPython.core.display.Javascript object>

In [6]:
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


Keep only positive and negative tweets (so you exclude neutrals). What is the percentage of positive/negative tweets?


In [7]:
df = df[df.sentiment!="neutral"]

Retrieve the list of English stopwords from NLTK, and copy it into a stopwordsenglish list.


In [8]:
stopwordsenglish  = nltk.corpus.stopwords.words("english")
type(stopwordsenglish)

list

Create a lemma function that takes a text (str) as a parameter and returns a text (str) of tokens after applying a stemmer or a lemmatizer, separated by spaces.

In [9]:
nlp = spacy.load('en_core_web_sm')

def lem_tokenize(text):
  lemmas = []
  sent_tokens = nlp(text)
  for token in sent_tokens:
    lemmas.append(token.lemma_)
  return " ".join(lemmas)
lem_tokenize("You are better when I am well.")

'-PRON- be well when -PRON- be well .'

Apply this function lemma to the text column of your DataFrame. Store the result in a new lemma column of the DataFrame. (The processing can take 2 or 3 minutes).

In [10]:
df['lemma'] = df['text'].apply(lem_tokenize)

In [11]:
df.head()

,textID,text,selected_text,sentiment,lemma
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,Sooo SAD -PRON- will miss -PRON- here in San...
2,088c60f138,my boss is bullying me...,bullying me,negative,-PRON- boss be bully -PRON- ...
3,9642c003ef,what interview! leave me alone,leave me alone,negative,what interview ! leave -PRON- alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"son of * * * * , why couldn`t -PRON- put -PR..."
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive,2 a.m. feeding for the baby be fun when -PRON-...


Copy the lemma column into a Series X, and the sentiment column into a Series y. Apply a train test split with the random_state = 32.


In [12]:
X = pd.Series(df["lemma"])
y = pd.Series(df["sentiment"])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=32)

<IPython.core.display.Javascript object>

Create a vectorizer model with scikit-learn using the Countvectorizer method. Train your model on X_train, then create a matrix of features X_train_CV. Create the X_test_CV matrix without re-training the model. The format of the X_test_CV matrix should be 4091x13838 with 44274 stored elements.

In [14]:
vectorizer = CountVectorizer()


In [15]:
X_train_CV = vectorizer.fit_transform(X_train)

In [16]:
X_test_CV = vectorizer.transform(X_test)

In [17]:
X_test_CV

<4091x13838 sparse matrix of type '<class 'numpy.int64'>'
	with 44274 stored elements in Compressed Sparse Row format>

Using X_train_CV and y_train, please run a logistic regression to classify tweets. You should get a score of 0.87 on the X_test_CV test set.


In [21]:
lr = LogisticRegression(max_iter=150)

In [22]:
lr_model = lr.fit(X_train_CV, y_train)

In [23]:
lr_model.score(X_train_CV, y_train), lr_model.score(X_test_CV, y_test)

(0.9590938722294654, 0.8719139574676118)

Repeat steps 7 and 8, adding the stop_words parameter to the CountVectorizer, using your stopwordsenglish list from step 3. Is the score very different? Is the number of elements stored in the sparse matrix very different?

In [25]:
lr_ = LogisticRegression(max_iter=200)
vectorizer_ = CountVectorizer(stop_words=stopwordsenglish)

X_train_CV_ = vectorizer_.fit_transform(X_train)
X_test_CV_ = vectorizer_.transform(X_test)


lr_model_ = lr_.fit(X_train_CV_, y_train)
lr_model_.score(X_train_CV_, y_train), lr_model_.score(X_test_CV_, y_test)

(0.9536342894393742, 0.8631141530188218)

In [26]:
X_test_CV_

<4091x13712 sparse matrix of type '<class 'numpy.int64'>'
	with 28842 stored elements in Compressed Sparse Row format>

Test the model using TfidfVectorizer instead of Countvectorizer, and repeat steps 7 and 8. Which preprocessing method gives you the best results on the test set? Are the scores very different on this dataset?

In [27]:
lr_2 = LogisticRegression(max_iter=200)
tvectorizer_ = TfidfVectorizer(stop_words=stopwordsenglish)

X_train_CV_2 = tvectorizer_.fit_transform(X_train)
X_test_CV_2 = tvectorizer_.transform(X_test)


lr_model_2 = lr_2.fit(X_train_CV_2, y_train)
lr_model_2.score(X_train_CV_2, y_train), lr_model_2.score(X_test_CV_2, y_test)

(0.9275586701434159, 0.8655585431434857)

In [28]:
X_train_CV_2

<12272x13712 sparse matrix of type '<class 'numpy.float64'>'
	with 95192 stored elements in Compressed Sparse Row format>

Bonus step: try to display 10 tweets that were badly predicted (false positive or false negative). Would you have done better than the algorithm?


In [29]:
X_all = tvectorizer_.transform(X)
preds = lr_model_2.predict(X_all)

In [30]:
df['preds'] = preds

In [31]:
df[df['sentiment'] != df['preds']].head(10)

,textID,text,selected_text,sentiment,lemma,preds
17,919fa93391,i`ve been sick for the past few days and thus...,sick,negative,"i`ve be sick for the past few day and thus ,...",positive
76,35f393a245,"WOW, i AM REALLY MiSSiN THE FAM(iLY) TODAY. BA...",BADDD.,negative,"wow , i am really missin the fam(ily ) today ....",positive
103,fd9fb4f1ad,i realy wanted to go out cause its so nice but...,busy,negative,i realy want to go out cause -PRON- so nice bu...,positive
108,561b44a42f,have a safe trip joshy poo.......you`ll knock...,safe,positive,have a safe trip joshy poo ....... you`ll kn...,negative
124,f0460d611d,not well,not well,negative,not well,positive
125,6649f3558c,Not a prob hun,Not a prob,positive,not a prob hun,negative
148,ae71041e21,Candle wax is very enjoyable.,enjoyable.,positive,candle wax be very enjoyable .,negative
149,11c10e3fc5,She`s unassuming and unpretentious. She`s jus...,endearing-,positive,She`s unassuming and unpretentious . She`s j...,negative
178,9d3a1e0269,really hopes her car`s illness is not terminal...,illness,negative,really hope -PRON- car`s illness be not termin...,positive
182,2bf5c7dba7,I VOTED!!! do u have a personal myspace? i ke...,i <3 you. u helped me thru the hrdest time of ...,positive,-PRON- vote ! ! ! do u have a personal myspa...,negative
